# PaddleOCR + LayoutLM 发票识别模型模型训练、评估和优化

### 🎯 学习目标
> 1. 数据增强
> 2. 模型调参
> 3. 损失函数与优化
> 4. 后处理优化 (正则表达式，泛化，其他模型)
> 5. 错误分析
> 6. 模型集成

## 0. 系统架构设计

我们的发票识别系统采用多阶段处理架构：

```
PDF文档 → 图像转换 → OCR文本提取 → 布局分析 → LayoutLM处理 → 信息提取 → 结构化输出
```

**核心组件：**
- **PaddleOCR**: 负责文本检测和识别
- **LayoutLM**: 理解文档布局和语义关系
- **后处理模块**: 字段验证和格式化

## 1. 数据增强

###  PDF到图像转换

**例： data_augmentation.py**

**关键功能：**
- PDF文档解析和图像转换
- 图像质量增强（去噪、对比度调整、锐化）
- 训练数据标注格式转换
- 轻微旋转（±5度）
- 亮度调整（0.8-1.2倍）
- 对比度变化
- 高斯噪声
- 模糊处理

**图像增强技术：**
1. **CLAHE对比度增强**: 改善文档可读性
2. **形态学操作**: 去除噪点和伪影
3. **锐化滤波**: 提升文字边缘清晰度

In [ ]:
# 在数据预处理阶段添加数据增强
from data_augmentation import InvoiceDataAugmentation

augmenter = InvoiceDataAugmentation()
# 为每个训练样本生成2-3个增强版本
augmented_images = augmenter.augment_image(image, num_augmentations=3)

## 2. 模型调参

**例： layoutlm_training.py**

### 数据集构建


<mcfile name="layoutlm_dataset.py" path="/Users/xiaotingzhou/Documents/Lectures/AI_OCR/layoutlm_dataset.py"></mcfile> 实现了专门的数据集类：

**BIO标注体系：**
- `B-InvoiceNo`: 发票号码开始
- `I-InvoiceNo`: 发票号码内部
- `B-InvoiceDate`: 日期开始
- `I-InvoiceDate`: 日期内部
- `O`: 其他标签

### 模型训练配置


**训练参数：**
- 学习率: 5e-5
- 批次大小: 4
- 训练轮数: 10
- 权重衰减: 0.01

**🔍 各参数详细说明：**

1. num_train_epochs=10 : 完整训练数据集的遍历次数
2. per_device_train_batch_size=4 : 每个GPU/CPU设备上的训练样本批次大小
3. per_device_eval_batch_size=4 : 验证时的批次大小
4. warmup_steps=500 : 学习率从0逐渐增加到设定值的步数
5. weight_decay=0.01 : L2正则化系数，防止过拟合
6. learning_rate=5e-5 : 初始学习率
7. evaluation_strategy="epoch" : 每个epoch结束后进行评估
8. save_strategy="epoch" : 每个epoch结束后保存模型
9. metric_for_best_model="eval_f1" : 使用F1分数作为最佳模型选择标准

**评估指标：**
- F1分数
- 精确率
- 召回率
- 字段级准确率

### 🚀 性能优化建议

#### 1. 学习率优化

In [ ]:
# 当前配置
learning_rate=5e-5

# 优化建议：使用学习率查找或分层学习率
learning_rate=3e-5,  # 降低学习率，提高稳定性
# 或者使用余弦退火
lr_scheduler_type="cosine",

#### 2. 批次大小优化

In [ ]:
# 当前配置
per_device_train_batch_size=4

# 优化建议：根据GPU内存调整
per_device_train_batch_size=8,  # 如果内存允许，增加批次大小
gradient_accumulation_steps=2,  # 梯度累积，模拟更大批次

#### 3. 训练轮数与早停优化

In [ ]:
# 当前配置
num_train_epochs=10

# 优化建议：增加训练轮数并使用早停
num_train_epochs=20,
# 在Trainer中添加早停回调
callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]

#### 4. 评估策略优化

In [ ]:
# 当前配置
evaluation_strategy="epoch"

# 优化建议：更频繁的评估
evaluation_strategy="steps",
eval_steps=200,  # 每200步评估一次
save_steps=200,  # 每200步保存一次

#### 5. 正则化优化

In [ ]:
# 当前配置
weight_decay=0.01

# 优化建议：调整权重衰减
weight_decay=0.005,  # 减少正则化强度
# 添加dropout
dropout=0.1,  # 在模型初始化时设置

### 3. 损失函数与优化

**交叉熵损失：**
```python
loss_fct = CrossEntropyLoss()
loss = loss_fct(logits.view(-1, num_labels), labels.view(-1))
```

**学习率调度：**
- 线性预热策略
- 余弦退火调度
- 早停机制

### 1. 线性预热策略 (Linear Warmup)
#### 原理
线性预热策略是一种学习率调度技术，在训练初期逐渐增加学习率，而不是直接使用目标学习率。

#### 工作机制
- 初始阶段 ：从很小的学习率（如0或目标学习率的1%）开始
- 预热期 ：在预设的步数内线性增加学习率至目标值
- 正常训练 ：达到目标学习率后按正常策略训练
#### 优势
- 避免梯度爆炸 ：防止训练初期大学习率导致的不稳定
- 更好的收敛 ：让模型参数平稳地适应训练过程
- 提高最终性能 ：特别适用于大批次训练和Transformer模型

In [ ]:
def linear_warmup_scheduler(step, warmup_steps, base_lr, target_lr):
    if step < warmup_steps:
        return base_lr + (target_lr - base_lr) * step / warmup_steps
    return target_lr

### 2. 余弦退火调度 (Cosine Annealing)
#### 原理
余弦退火调度使用余弦函数来调整学习率，实现平滑的学习率衰减。

#### 工作机制
- 数学公式 ： lr = lr_min + (lr_max - lr_min) * (1 + cos(π * epoch / T_max)) / 2
- 周期性变化 ：学习率按余弦曲线从最大值平滑降至最小值
- 重启机制 ：可选择性地重启调度周期
#### 优势
- 平滑衰减 ：避免突然的学习率变化
- 逃离局部最优 ：周期性重启帮助跳出局部最优解
- 更好的泛化 ：渐进式衰减有利于模型泛化
#### 实现示例

In [1]:
import math

def cosine_annealing_lr(epoch, T_max, lr_max, lr_min=0):
    return lr_min + (lr_max - lr_min) * (1 + math.cos(math.pi * epoch / T_max)) / 2

### 3. 早停机制 (Early Stopping)
#### 原理
早停机制通过监控验证集性能，在模型开始过拟合时提前终止训练。

#### 工作机制
- 监控指标 ：通常监控验证损失或验证准确率
- 耐心参数 ：设置连续多少个epoch性能不改善就停止
- 最佳模型保存 ：保存验证性能最好的模型权重
#### 优势
- 防止过拟合 ：避免模型在训练集上过度拟合
- 节省计算资源 ：减少不必要的训练时间
- 自动化训练 ：无需手动判断何时停止训练
#### 实现示例

![Transformer Model Architecture](https://miro.medium.com/v2/resize:fit:1134/format:webp/0*z19dbRlkgocQYn6t.png)

In [2]:
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_loss = None
        self.counter = 0
        self.best_weights = None
    
    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(model)
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.save_checkpoint(model)
        else:
            self.counter += 1
        
        if self.counter >= self.patience:
            if self.restore_best_weights:
                model.load_state_dict(self.best_weights)
            return True
        return False
    
    def save_checkpoint(self, model):
        self.best_weights = model.state_dict().copy()

## 综合应用策略
### 组合使用
这三种策略通常组合使用以获得最佳效果：

In [ ]:
# 完整的训练循环示例
def train_with_optimization_strategies(model, train_loader, val_loader, epochs=100):
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    
    # 早停机制
    early_stopping = EarlyStopping(patience=10)
    
    # 学习率调度器（结合预热和余弦退火）
    warmup_steps = len(train_loader) * 3  # 前3个epoch预热
    total_steps = len(train_loader) * epochs
    
    for epoch in range(epochs):
        model.train()
        for step, batch in enumerate(train_loader):
            # 线性预热 + 余弦退火
            current_step = epoch * len(train_loader) + step
            if current_step < warmup_steps:
                lr = linear_warmup_scheduler(current_step, warmup_steps, 1e-7, 1e-4)
            else:
                lr = cosine_annealing_lr(current_step - warmup_steps, 
                                       total_steps - warmup_steps, 1e-4, 1e-6)
            
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            
            # 训练步骤
            optimizer.zero_grad()
            loss = model(batch)
            loss.backward()
            optimizer.step()
        
        # 验证和早停检查
        val_loss = validate(model, val_loader)
        if early_stopping(val_loss, model):
            print(f"Early stopping at epoch {epoch}")
            break

### 🔧 模型架构优化 （dropout）

In [ ]:
# 在模型初始化时添加更多配置
self.model = LayoutLMForTokenClassification.from_pretrained(
    model_name,
    num_labels=self.num_labels,
    hidden_dropout_prob=0.1,        # 隐藏层dropout
    attention_probs_dropout_prob=0.1, # 注意力dropout
    classifier_dropout=0.1,          # 分类器dropout
)

### 4. 后处理优化（正则表达式：Regular Expressions）
**例： tesseract_extraction.py**

**日期标准化：**
```python
def normalize_date(self, date_text):
    chinese_pattern = r'(\d{4})年(\d{1,2})月(\d{1,2})日'
    match = re.search(chinese_pattern, date_text)
    if match:
        year, month, day = match.groups()
        return f"{year}年{month.zfill(2)}月{day.zfill(2)}日"
    return date_text.strip()
```

**金额提取：**
```python
def normalize_amount(self, amount_text):
    numbers = re.findall(r'\d+\.?\d*', amount_text)
    return numbers[0] if numbers else "0"
```

## 模型评估与优化

### 评估框架

<mcfile name="model_evaluation.py" path="/Users/xiaotingzhou/Documents/Lectures/AI_OCR/model_evaluation.py"></mcfile> 提供了全面的评估工具：

**评估维度：**
- 字段级准确率
- 整体系统性能
- 错误类型分析
- 置信度分布

## 5. 错误分析

**预期性能指标：**
- 发票号码识别: 95%+
- 日期识别: 90%+
- 金额识别: 95%+
- 整体准确率: 92%+


**常见错误类型：**
1. **OCR识别错误**: 字符混淆（如0和O）
2. **布局理解错误**: 字段边界不准确
3. **后处理错误**: 格式转换失败

**改进策略：**
- 增加训练数据多样性
- 优化图像预处理
- 改进后处理规则

**数据平衡**: 各字段样本数量均衡： Accuracy, F1, ROC_AUC, Percision,....

#### 📈 监控与调试

1. 使用性能监控器 ：
   
   - 利用 `performance_monitor.py` 监控训练性能
2. 学习率调度 ：
   
   - 使用学习率查找器找到最优学习率
   - 监控验证损失，及时调整学习率
3. 梯度监控 ：
   
   - 监控梯度范数，防止梯度爆炸或消失
### 🎯 针对文本提取的特殊优化
1. 标签平衡 ：处理标签不平衡问题
2. 序列长度优化 ：根据实际文档调整 max_length
3. 位置编码 ：确保边界框坐标正确归一化
4. 多尺度训练 ：使用不同分辨率的图像进行训练
通过这些优化策略，您可以显著提升LayoutLM模型在发票文本提取任务上的性能和准确率。建议逐步应用这些优化，并通过验证集监控效果。

### 6. 模型优化策略：模型集成

1. **渐进式训练**: 从简单到复杂的训练策略
2. **集成学习**: 多模型融合提升性能

# 集成学习在发票信息提取中的应用教学 (Ensemble Learning)
## 概述
集成学习通过组合多个模型来提高预测性能，在发票信息提取任务中可以显著提升准确率和鲁棒性。本教学将详细介绍Bagging、Boosting和Stacking三种方法在发票OCR中的应用

## 1. Bagging（Bootstrap Aggregating 自助聚集）
### 理论基础
Bagging通过自助采样创建多个训练子集，训练多个模型并平均预测结果，减少方差

	•	原理：从原始训练集通过有放回抽样（Bootstrap）生成多个子训练集；分别训练多个独立（通常是决策树）模型，再对它们输出进行平均（回归）或投票（分类）汇总 ￼。
	•	好处：极大减少模型的方差，抑制过拟合，提升鲁棒性；典型应用如随机森林。
	•	缺点：仅对减小方差有效，对偏差（bias）改进有限，如果基础模型关联性太高，效果有限。

## How Does Bagging Work?

![How Does Bagging Work?](https://i0.wp.com/spotintelligence.com/wp-content/uploads/2024/03/bagging-1024x576.webp?resize=1024%2C576&ssl=1)

### ensemble_bagging_invoice.py

## 2. Boosting（自适应提升） XGBoost?
### 理论基础
Boosting通过序列化训练弱学习器，每个新模型专注于前一个模型的错误，逐步提升性能。

### ensemble_boosting_invoice.py

	•	原理：顺序训练一系列弱学习器，每个新模型关注纠正前一个模型的错误。常见算法 AdaBoost（自适应提升）或 Gradient Boosting（梯度提升）。
	•	好处：显著减少偏差，提高模型准确性；对弱模型进行增强。
	•	缺点：容易过拟合，尤其当迭代太多时；对噪声敏感，计算复杂度和时间也较大。

## How Does Boosting Work?

![How Does Boosting Work?](https://i0.wp.com/spotintelligence.com/wp-content/uploads/2024/03/boosting-1024x576.webp?resize=1024%2C576&ssl=1)

## 3. Stacking（堆叠泛化）
### 理论基础
Stacking使用元学习器来学习如何最优地组合基础模型的预测结果。

### ensemble_stacking_invoice.py

	•	原理：并行训练多种不同类型的基础模型（如决策树、支持向量机等），然后将它们的预测作为特征输入给“元模型”（meta-model），由元模型学习最佳组合方式输出最终预测 ￼。
	•	好处：能有效整合多种模型优势，进一步提高表现；适合结构多样的基学习器。
	•	缺点：实现更复杂；训练过程耗时，计算资源要求高；解释性变差。


## How does Stacking Work?

![How does Stacking Work?](https://i0.wp.com/spotintelligence.com/wp-content/uploads/2024/03/stacking-1024x576.webp?resize=1024%2C576&ssl=1)

### 🔧 如何用 Ensemble Learning ？

我们可以在多个层面使用集成策略：

#### 🧩 1. 模型级集成（Model Ensemble）
- 同时训练多个 LayoutLM 类模型（如 LayoutLMv2, LayoutXLM, Donut, TrOCR 等）
- 对它们预测的结构化字段进行：
- 多数投票（majority voting）
- 置信度加权平均（confidence-weighted averaging）
- stacking：用一个 meta-model 融合多个模型输出

📌 适用场景：
- 每个模型可能对不同字段有不同优势（如金额 vs 日期 vs 公司名）
- 某些字段识别困难时，可融合多个模型降低误差

#### 🧩 2. 输入数据增强 + Bagging

- 对每张发票图片做轻微数据增强（如轻度旋转、缩放、亮度变化）
- 生成多个 OCR 结果输入 LayoutLM
- 对多个版本的结构化输出进行 Bagging（投票、平均）

📌 优点：
- 提高系统对噪声/格式变化的鲁棒性
- 模拟测试集多样性，提高泛化能力


#### 🧩 3. 多阶段 Boosting
- 第一阶段：用轻量模型（如规则模板、keyword match）快速粗筛字段
- 第二阶段：再用 LayoutLM 微调模型精细修正
- 第三阶段：用规则/置信度过滤结果（如金额必须为数字、日期必须有年）

📌 类似 Boosting 中「一阶段纠错一阶段」，逐层纠偏。


#### 集成学习的整体优势与注意

> 优势
	- 综合多个模型提升性能、减少错误（bias + variance）。
	- 在大多数任务中显著优于单一模型 ￼。
- 注意事项
	- 组合的模型必须具有多样性，才能获得协同效应()。
	- Boosting 的连锁误差与噪声敏感性需要谨慎控制。
	- Stacking 的多层结构使得调试、解释与部署更具挑战性。